# 0.0. IMPORTS

In [1]:
import pandas   as pd
import numpy    as np
import psycopg2 as pg
import seaborn  as sns


import pandas.io.sql          as psql
from matplotlib import pyplot as plt

from IPython.core.display  import HTML

import warnings

## 0.1. Helper Functions

In [67]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
    
warnings.filterwarnings( 'ignore' )
pd.options.display.float_format = '{:.2f}'.format

In [68]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 0.2. Loading Data

In [4]:
# Credentials
host =     'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port =      5432
database = 'comunidadedsdb'
username = 'member'
pwd =      'cdspa'



# Connecting to database
conn = pg.connect(  user=username,
                    password=pwd,
                    host=host,
                    port=port,
                    database=database )



cursor = conn.cursor()

In [5]:
# Query Schemas
query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""

cursor.execute( query_schema )

print(cursor.fetchall())

[('pg_toast',), ('pg_temp_1',), ('pg_toast_temp_1',), ('pg_catalog',), ('information_schema',), ('public',), ('pa004',), ('pg_temp_4',), ('pg_toast_temp_4',), ('pa005',), ('pg_temp_9',), ('pg_toast_temp_9',)]


In [6]:
# Query Tables
query_tables = """
SELECT * FROM pa004.users u LEFT JOIN pa004.vehicle v on (u.id = v.id)
                            LEFT JOIN pa004.insurance i on (u.id = i.id) 
"""

# Defining raw dataset
df_raw_table = pd.read_sql(query_tables, conn)


# Closing cursor and connction
cursor.close()
conn.close()

In [7]:
df_raw_table.head()

,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,7,Male,23,11.0,152.0,7,1,< 1 Year,Yes,7,0,23367.0,249,0
1,13,Female,41,15.0,14.0,13,1,1-2 Year,No,13,1,31409.0,221,0
2,18,Female,25,35.0,152.0,18,1,< 1 Year,No,18,1,46622.0,299,0
3,31,Female,26,8.0,160.0,31,1,< 1 Year,No,31,0,2630.0,136,0
4,39,Male,45,8.0,124.0,39,1,1-2 Year,Yes,39,0,42297.0,264,0


# 1.0. Data Description

In [49]:
df1 = df_raw_table.copy()

In [50]:
df1.columns

Index(['id', 'gender', 'age', 'region_code', 'policy_sales_channel', 'id',
       'driving_license', 'vehicle_age', 'vehicle_damage', 'id',
       'previously_insured', 'annual_premium', 'vintage', 'response'],
      dtype='object')

In [53]:
# Removing duplicated columns
df1 = df1.loc[:,~df1.columns.duplicated()]

## 1.1. Data Dimension

In [54]:
print(f"Number of Rows: {df1.shape[0]}")
print(f"Number of Columns: {df1.shape[1]}")

Number of Rows: 381109
Number of Columns: 12


## 1.2. Data Types

In [19]:
df1.dtypes

id                        int64
gender                   object
age                       int64
region_code             float64
policy_sales_channel    float64
id                        int64
driving_license           int64
vehicle_age              object
vehicle_damage           object
id                        int64
previously_insured        int64
annual_premium          float64
vintage                   int64
response                  int64
dtype: object

## 1.3. Check NA

In [58]:
df1.isnull().sum()

id                      0
gender                  0
age                     0
region_code             0
policy_sales_channel    0
driving_license         0
vehicle_age             0
vehicle_damage          0
previously_insured      0
annual_premium          0
vintage                 0
response                0
dtype: int64

## 1.4. Data Descriptive

In [60]:
num_attributes = df1.select_dtypes( include=['int64', 'float64'] )
cat_attributes = df1.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'] )

In [69]:
# Central Tendency - Mean and Median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( min ) ).T
d3 = pd.DataFrame( num_attributes.apply( max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

# Concatenate
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

In [70]:
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,id,1.00,381109.00,381108.00,190555.00,190555.00,110016.69,-0.00,-1.20
1,age,20.00,85.00,65.00,38.82,36.00,15.51,0.67,-0.57
2,region_code,0.00,52.00,52.00,26.39,28.00,13.23,-0.12,-0.87
3,policy_sales_channel,1.00,163.00,162.00,112.03,133.00,54.20,-0.90,-0.97
4,driving_license,0.00,1.00,1.00,1.00,1.00,0.05,-21.60,464.35
5,previously_insured,0.00,1.00,1.00,0.46,0.00,0.50,0.17,-1.97
6,annual_premium,2630.00,540165.00,537535.00,30564.39,31669.00,17213.13,1.77,34.00
7,vintage,10.00,299.00,289.00,154.35,154.00,83.67,0.00,-1.20
8,response,0.00,1.00,1.00,0.12,0.00,0.33,2.30,3.30


# 2.0. Feature Engineering

In [71]:
df2 = df1.copy()

In [76]:
df2.head()

,id,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,7,Male,23,11.00,152.00,1,below_1_year,1,0,23367.00,249,0
1,13,Female,41,15.00,14.00,1,between_1_2_year,0,1,31409.00,221,0
2,18,Female,25,35.00,152.00,1,below_1_year,0,1,46622.00,299,0
3,31,Female,26,8.00,160.00,1,below_1_year,0,0,2630.00,136,0
4,39,Male,45,8.00,124.00,1,between_1_2_year,1,0,42297.00,264,0


In [75]:
# vehicle age
df2['vehicle_age'] = df2['vehicle_age'].apply( lambda x: 'over_2_years' if x =='> 2 Years' else 'between_1_2_year' if x == '1-2 Year' else 'below_1_year') 

# vehicle damage
df2['vehicle_damage'] = df2['vehicle_damage'].apply( lambda x: 0 if x == "No" else 1 ) 

# 3.0. Data Filtering

In [77]:
df3 = df2.copy()

# 4.0. EDA

# 5.0. Feature Selection

# 6.0. Data preparation

# 7.0. Machine Learning Model